# 🎤 Text to Speech (TTS) & Speech to Text (STT) 실습 강의 노트

- OpenAI TTS 데모: [OpenAI.fm](https://openai.fm)

## 1) 환경 준비 및 API 키 설정

In [ ]:
pip install openai

- Colab 메뉴 `보안 비밀(열쇠 아이콘)`에 `OPENAI_API_KEY`를 저장해 두었다면 자동으로 불러옵니다.
- 아래의 코드에서 오류가 난다면 API 키의 오류일 가능성이 높습니다.

In [ ]:
import os

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception:
    print("Colab 메뉴의 보안 비밀(열쇠 아이콘)에서 환경 변수가 제대로 저장되었는지 확인해주세요.")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API 키 설정 완료")

In [ ]:
from openai import OpenAI, AsyncOpenAI
from pathlib import Path

client = OpenAI()
async_client = AsyncOpenAI()
print("✅ 클라이언트 생성 완료")


## 2) Text To Speech (TTS)

TTS는 텍스트를 오디오로 변환하는 모델입니다.

OpenAI API에는 11개의 내장 음성이 포함되어 있으며 다음과 같이 사용할 수 있습니다:
- 작성된 블로그 포스트를 나레이션하기
- 다국어로 음성 오디오 생성하기
- 스트리밍을 사용한 실시간 오디오 출력 제공하기

### 주요 파라미터

**model**: 사용 가능한 TTS 모델 중 하나를 지정합니다. 
- `tts-1`, `tts-1-hd`, `gpt-4o-mini-tts` 중 선택 가능합니다.
- 실습 모델: gpt-4o-mini-tts

**input**: 오디오를 생성할 텍스트입니다. 
- 최대 길이는 4096자입니다.

**voice**: 오디오를 생성할 때 사용할 음성입니다. 
- 지원되는 음성은 `alloy`, `ash`, `ballad`, `coral`, `echo`, `fable`, `onyx`, `nova`, `sage`, `shimmer`, `verse` 입니다. 
- 음성의 미리듣기는 [OpenAI.fm](https://openai.fm)에서 확인할 수 있습니다.

**instructions**: 생성된 오디오의 음성을 추가 지시사항으로 제어합니다. 
- 억양, 감정, 속도 등을 지정할 수 있습니다. 
- `tts-1` 또는 `tts-1-hd` 모델에서는 작동하지 않습니다.

**response_format**: 오디오 출력 형식을 지정합니다. 
- 지원되는 형식은 `mp3`(기본값), `opus`, `aac`, `flac`, `wav`, `pcm` 입니다.

**speed**: 생성된 오디오의 속도입니다. 
- 0.25에서 4.0 사이의 값을 선택할 수 있습니다. 
- 기본값은 1.0입니다.

In [ ]:
from pathlib import Path
from openai import OpenAI

client = OpenAI()
speech_file_path = Path(__file__).parent / "speech.mp3" # 경로는 좀 체크 해봐야함
#speech_file = Path("speech.mp3")
#speech_file.unlink(missing_ok=True)

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="와! 오늘 같은 날은 LLM 실습 배우기에 완전히 최고의 날이에요! 정말 신나요!",
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file_path)

> **설명 포인트**
> - `with_streaming_response.create` 컨텍스트를 사용하면 서버로부터 오는 오디오 스트림을 실시간으로 파일에 쓸 수 있습니다.
> - 음성 합성 요청마다 `voice`, `instructions`를 통해 억양/감정/속도 등을 제어할 수 있습니다.



### 스트리밍 재생 및 실시간 활용
TTS 응답을 바로 재생하려면 스트리밍 응답과 로컬 플레이어를 결합할 수 있습니다.


In [ ]:
import asyncio
from openai.helpers import LocalAudioPlayer

async def play_streaming_tts(text: str):
    async with async_client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input=text,
        instructions="Speak in a cheerful and positive tone.",
        response_format="pcm",
    ) as response:
        await LocalAudioPlayer().play(response)

# asyncio.run(play_streaming_tts("와! 오늘 같은 날은 LLM 실습 배우기에 완전히 최고의 날이에요! 정말 신나요!"))


> **설명 포인트**
> - `response_format="wav"` 또는 `"pcm"`을 사용하면 지연을 줄이고 빠르게 스트리밍 재생할 수 있습니다.
> - 웹/모바일 앱에서는 스트리밍 응답을 스피커로 직접 송출하거나 음성 채팅에 활용할 수 있습니다.



## 3) Speech To Text (SST)

## 참고자료
- https://platform.openai.com/docs/api-reference/audio/createSpeech
- https://platform.openai.com/docs/guides/text-to-speech
- https://openai.fm